In [ ]:
from collections import Counter
sorted([(name, Counter(name[0].lower().translate(dict.fromkeys(map(ord, 'aeiounrst .-'), None)))) for name in Card.objects.values_list('name')], key=lambda x: max(x[1].values() or [0]), reverse=True)


In [ ]:
from collections import Counter
Counter(''.join([name[0].lower() for name in Card.objects.values_list('name')]))

In [ ]:
from cards.models import *
print(f'Cards: {Card.objects.count()}')

In [ ]:
print(f'CardPrintings: {CardPrinting.objects.count()}')

In [ ]:
print(f'CardLocalisations: {CardLocalisation.objects.count()}')

In [ ]:
print(f'PhysicalCards: {PhysicalCard.objects.count()}')

In [ ]:
print(f'UserOwnedCards: {UserOwnedCard.objects.filter(owner=User.objects.get(username="Liam")).count()}')

In [ ]:
from django.db.models import Sum
print(f"My Cards: {UserOwnedCard.objects.filter(owner=User.objects.get(username='Liam')).aggregate(Sum('count'))}")

In [ ]:
from django.db import connection
with connection.cursor() as cursor:
    cursor.execute("""
SELECT SUM(reltuples) rowcount
FROM pg_class C
LEFT JOIN pg_namespace N ON (N.oid = C.relnamespace)
WHERE nspname NOT IN ('pg_catalog', 'information_schema') 
AND relkind='r' 
""")
    result = cursor.fetchone()
    
print(result)

In [3]:
for card in Card.objects.filter(is_token=True).all():
    card.name = f"{card.name} ({card.scryfall_oracle_id.split('-')[0]})"
    card.save()

In [6]:
Card.objects.filter(id__in=[29142, 29143, 29145, 29146, 29147, 29148, 29149, 29151, 29152, 29154, 29155, 29156, 29157, 29158, 29159, 29170, 29117, 29118, 29131, 29132, 29141, 29144, 29150, 29153, 29160, 29169]).delete()

(130,
 {'cards.Card_links': 0,
  'cards.DeckCard': 0,
  'cards.CardLocalisation_physical_cards': 26,
  'cards.CardImage': 26,
  'cards.CardRuling': 0,
  'cards.CardLegality': 0,
  'cards.CardTag_cards': 0,
  'cards.CardLocalisation': 26,
  'cards.CardPrinting': 26,
  'cards.Card': 26})

In [4]:
from django.db.models import F, Func
from django.db.models import IntegerField
from django.db.models.functions import Cast
#Card.objects.filter(type__icontains="Legend", num_power=F('num_toughness'), mana_value=F('num_power'), mana_value__gte=1, colour_flags=F('colour_flags'))
#Card.objects.filter(colour_flags=F('colour_flags').bitand(Card.colour_flags.white))
# Card.objects.filter(colour_flags=F('colour_identity_flags'))
#Card.objects.filter(colour_flags=Cast('colour_identity_flags', output_field=IntegerField))
#Card.objects.annotate(colour_identity_flags_int=Cast('colour_identity_flags', output_field=IntegerField)filter(colour_flags=Func(F('colour_identity_flags'), function='ROUND'))
Card.objects.annotate(colour_identity_flags_int=Cast('colour_identity_flags', output_field=IntegerField))

TypeError: cast_db_type() missing 1 required positional argument: 'connection'

In [16]:
Card.objects.filter(colour_flags__lt=F('colour_flags') + F('colour_flags').bitand(4)).filter(name__icontains='boros')

<QuerySet [<Card: Evil Boros Charm>]>

In [4]:
str(Card.objects.filter(colour_flags=Card.colour_flags.white).query)

'SELECT "cards_card"."id", "cards_card"."name", "cards_card"."display_name", "cards_card"."cost", "cards_card"."mana_value", "cards_card"."face_mana_value", "cards_card"."colour_flags", "cards_card"."colour_identity_flags", "cards_card"."colour_indicator_flags", "cards_card"."colour_count", "cards_card"."colour_identity_count", "cards_card"."colour_sort_key", "cards_card"."colour_weight", "cards_card"."type", "cards_card"."subtype", "cards_card"."power", "cards_card"."num_power", "cards_card"."toughness", "cards_card"."num_toughness", "cards_card"."loyalty", "cards_card"."num_loyalty", "cards_card"."rules_text", "cards_card"."layout", "cards_card"."side", "cards_card"."hand_modifier", "cards_card"."num_hand_modifier", "cards_card"."life_modifier", "cards_card"."num_life_modifier", "cards_card"."is_reserved", "cards_card"."scryfall_oracle_id", "cards_card"."edh_rec_rank", "cards_card"."is_token" FROM "cards_card" WHERE "cards_card"."colour_flags" & 1 = 1'

In [23]:
Card.objects.filter(colour_count=F('num_power'), power__isnull=False).order_by('-num_power')

<QuerySet [<Card: Maelstrom Archangel>, <Card: Atogatog>, <Card: Karona, False God>, <Card: Sphinx of the Guildpact>, <Card: Cromat>, <Card: Horde of Notions>, <Card: Sliver Hivelord>, <Card: Breya, Etherium Shaper>, <Card: Atraxa, Praetors' Voice>, <Card: Sol, Advocate Eternal>, <Card: Radiant Kavu>, <Card: Akim, the Soaring Wind>, <Card: Gwendlyn Di Corci>, <Card: Xyris, the Writhing Storm>, <Card: Tetsuo Umezawa>, <Card: Kess, Dissident Mage>, <Card: Rhox War Monk>, <Card: Sidisi, Brood Tyrant>, <Card: Tayam, Luminous Enigma>, <Card: Kathril, Aspect Warper>, '...(remaining elements truncated)...']>

In [24]:
Card.objects.filter(colour_flags=F('num_power'))

ProgrammingError: can't adapt type 'Col'

In [3]:
CardImage.objects.filter(printed_language__card_printing__card__type='Ongoing Scheme').delete()

(0, {'cards.CardImage': 0})

In [6]:
for printing in CardPrinting.objects.filter(card__is_token=True):
    if printing.set.type != 'token':
        token_set = Set.objects.get(code='T' + printing.set.code)
        printing.set = token_set
        printing.save()

In [7]:
CardImage.objects.filter(printed_language__card_printing__card__is_token=True).delete()

(1440, {'cards.CardImage': 1440})

In [13]:
for no_printing_set in Set.objects.annotate(printing_count=Count('card_printings')).filter(printing_count=0):
    if no_printing_set.type == 'token':
        other_sets = Set.objects.filter(code=no_printing_set.code[1:])
        if other_sets.exists():
            print(no_printing_set, ', ', other_sets.first())
            #no_printing_set.card_printings.update(set=other_sets.first())
    no_printing_set.delete()
            

In [16]:
[(s.name, s.code) for s in Set.objects.filter(type='token').order_by('release_date')]

[('Pro Tour Collector Set Tokens', 'TPTC'),
 ('World Championship Decks 1997 Tokens', 'TWC97'),
 ('Unglued Tokens', 'TUGL'),
 ('World Championship Decks 1998 Tokens', 'TWC98'),
 ('World Championship Decks 1999 Tokens', 'TWC99'),
 ('World Championship Decks 2000 Tokens', 'TWC00'),
 ('Magic Player Rewards 2001 Tokens', 'TMPR'),
 ('World Championship Decks 2001 Tokens', 'TWC01'),
 ('Magic Player Rewards 2002 Tokens', 'TPR2'),
 ('World Championship Decks 2002 Tokens', 'TWC02'),
 ('Magic Player Rewards 2003 Tokens', 'TP03'),
 ('World Championship Decks 2003 Tokens', 'TWC03'),
 ('Magic Player Rewards 2004 Tokens', 'TP04'),
 ('World Championship Decks 2004 Tokens', 'TWC04'),
 ('Coldsnap Tokens', 'TCSP'),
 ('Tenth Edition Tokens', 'T10E'),
 ('Lorwyn Tokens', 'TLRW'),
 ('Duel Decks: Elves vs. Goblins Tokens', 'TDD1'),
 ('Morningtide Tokens', 'TMOR'),
 ('Shadowmoor Tokens', 'TSHM'),
 ('Eventide Tokens', 'TEVE'),
 ('Shards of Alara Tokens', 'TALA'),
 ('Duel Decks: Jace vs. Chandra Tokens', 'TDD2'

In [11]:
Card.objects.filter(colour_flags=16).exclude(colour_flags=~16).filter(subtype="Sliver")

<QuerySet [<Card: Sliver Queen>, <Card: Spined Sliver>, <Card: Victual Sliver>, <Card: Darkheart Sliver>, <Card: Sliver Legion>, <Card: Virulent Sliver>, <Card: Sliver Hivelord>, <Card: The First Sliver>, <Card: Horned Sliver>, <Card: Muscle Sliver>, <Card: Megantic Sliver>, <Card: Brood Sliver>, <Card: Quick Sliver>, <Card: Root Sliver>, <Card: Firewake Sliver>, <Card: Gemhide Sliver>, <Card: Harmonic Sliver>, <Card: Might Sliver>, <Card: Spinneret Sliver>, <Card: Dormant Sliver>, '...(remaining elements truncated)...']>